In [ ]:
%run main.py
%load_ext autoreload
%autoreload 2

!mkdir -p {NAVEC_DIR} {MODEL_DIR} {PACK_DIR}
s3 = S3()

In [ ]:
if not exists(NAVEC):
    !wget {NAVEC_URL} -O {NAVEC}
    s3.download(S3_MODEL_SHAPE, MODEL_SHAPE)
    s3.download(S3_MODEL_ENCODER, MODEL_ENCODER)
    s3.download(S3_MODEL_HEAD, MODEL_HEAD)
    s3.download(S3_MODEL_REL, MODEL_REL)
    s3.download(S3_RELS_VOCAB, RELS_VOCAB)

In [ ]:
navec = Navec.load(NAVEC)

words_vocab = Vocab(navec.vocab.words)
shapes_vocab = Vocab([PAD] + SHAPES)
rels_vocab = Vocab.load(RELS_VOCAB)

word = NavecEmbedding(navec)
shape = Embedding(
    vocab_size=len(shapes_vocab),
    dim=SHAPE_DIM,
    pad_id=shapes_vocab.pad_id
)
emb = SyntaxEmbedding(word, shape)
encoder = SyntaxEncoder(
    input_dim=emb.dim,
    layer_dims=LAYER_DIMS,
    kernel_size=KERNEL_SIZE,
)
head = SyntaxHead(
    input_dim=encoder.dim,
    hidden_dim=encoder.dim // 2,
)
rel = SyntaxRel(
    input_dim=encoder.dim,
    hidden_dim=encoder.dim // 2,
    rel_dim=len(rels_vocab)
)
model = Syntax(emb, encoder, head, rel)
model.eval()

model.emb.shape.load(MODEL_SHAPE)
model.encoder.load(MODEL_ENCODER)
model.head.load(MODEL_HEAD)
model.rel.load(MODEL_REL)

In [ ]:
model = model.to_exec()
model = model.strip_navec()
arrays, model = model.separate_arrays()

In [ ]:
with DumpPack(PACK) as pack:
    meta = Meta(ID)
    pack.dump_meta(meta)
    
    pack.dump_model(model)
    pack.dump_arrays(arrays)

    pack.dump_vocab(words_vocab, WORD)
    pack.dump_vocab(shapes_vocab, SHAPE)
    pack.dump_vocab(rels_vocab, REL)

In [ ]:
syntax = api.Syntax.load(PACK)
syntax.navec(navec)
words = 'Опубликованы новые данные по заражению коронавирусом в Москве'.split()
markup = syntax(words)
show_syntax_markup(markup)

In [ ]:
s3.upload(PACK, S3_PACK)